In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm, pearsonr

import os
import time
import itertools

from glove.model import *

In [2]:
# import file names
files = os.listdir("data/SET3_Thirdtrial/")
files

['CDanc_CDmono12h_PROVIDET0.csv', 'CDevo_CDmono12h_PROVIDET0.csv']

# fit gLV models

In [3]:
exp_names = []
for file in files:
    # define strain name
    strain = file.split("_")[0]
    
    # import data
    df = pd.read_csv(f"data/SET3_Thirdtrial/{file}")
    df.sort_values(by=["Treatments", "Time"], inplace=True)
    
    # make sure that conditions have at least one measurement
    dfs = []
    for treatment, df_t in df.groupby("Treatments"):
        if df_t.shape[0] > 1:
            dfs.append(df_t)
    df = pd.concat(dfs)

    # determine species names 
    species = df.columns.values[2:]

    # instantiate gLV fit 
    model = gLV(species, df)

    # fit to data 
    model.fit()
    
    # list of parameter names 
    param_names = []
    for s1 in species:
        for s2 in species:
            param_names += [s1+"*"+s2]
    param_names = list(species) + param_names
    
    # plot parameter distribution
    n_species = len(species)
    Avec = model.params[n_species:]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:]

    plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])
    
    plt.suptitle(strain, fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("params/SET3/"+strain+".pdf")
    plt.close()
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    for j, param_name in enumerate(param_names):
        df[param_name]  = model.Ainv[:, j]
    df.to_csv("params/SET3/"+strain+".csv", index=False)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Total samples: 271, Initial regularization: 1.00e-03
Loss: 31.853, Residuals: -0.285
Loss: 16.700, Residuals: 0.148
Loss: 14.154, Residuals: 0.080
Loss: 10.455, Residuals: 0.044
Loss: 8.596, Residuals: 0.010
Loss: 8.314, Residuals: 0.030
Loss: 7.838, Residuals: 0.025
Loss: 7.272, Residuals: 0.032
Loss: 7.174, Residuals: 0.037
Loss: 7.010, Residuals: 0.034
Loss: 6.964, Residuals: 0.043
Loss: 6.878, Residuals: 0.039
Loss: 6.732, Residuals: 0.033
Loss: 6.698, Residuals: 0.041
Loss: 6.635, Residuals: 0.038
Loss: 6.530, Residuals: 0.031
Loss: 6.510, Residuals: 0.038
Loss: 6.473, Residuals: 0.035
Loss: 6.406, Residuals: 0.030
Loss: 6.402, Residuals: 0.036
Loss: 6.361, Residuals: 0.032
Loss: 6.287, Residuals: 0.026
Loss: 6.278, Residuals: 0.026
Loss: 6.200, Residuals: 0.020
Loss: 6.199, Residuals: 0.023
Loss: 6.186, Residuals: 0.023
Loss: 6.161, Residuals: 0.020
Loss: 6.133, Residuals: 0.018
Loss: 6.128, Residuals: 0.021
Loss: 6.090, Residuals: 0.015
Loss: 6.089, Residuals: 0.015
Loss: 6.087,

Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.952, Residuals: -0.008
Loss: 1.951, Residuals: -0.008
Loss: 1.951, Residuals: -0.008
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Loss: 1.951, Residuals: -0.009
Evidence -54.160
Updating hyper-parameters...
Total samples: 225, Updated regularization: 3.58e-01
Loss: 115.822, Residuals: -0.013
Loss: 105.910, Residuals: -0.003
Loss: 105.800, Residuals: -0.003
Optimization terminated successfully.
Evidence 3320.302
Updating hyper-parameters...
Total samples: 225, Updated regularization: 5.97e-01
Loss: 649.234, Residuals: -0.002
Optimization terminated successfully.
Evidence 4615.825
Updating hyper-pa

In [4]:
df

,Param name,Param value,Param stdv,Param p-value,CA,BT,BU,CD,BV,CS,...,DP*DP,DP*CH,CH*CA,CH*BT,CH*BU,CH*CD,CH*BV,CH*CS,CH*DP,CH*CH
0,CA,0.322340,0.022318,2.784668e-47,0.000498,0.000008,0.000022,0.000015,0.000019,-2.816653e-06,...,0.000009,0.000206,-0.000014,0.000007,-0.000017,0.000049,-0.000019,0.000442,0.000342,-0.000020
1,BT,0.888700,0.014790,0.000000e+00,0.000008,0.000219,0.000081,0.000005,0.000031,3.376683e-08,...,0.000006,0.000053,-0.000056,0.000185,0.000016,0.000023,0.000009,0.000025,0.000036,-0.000001
2,BU,0.922836,0.015662,0.000000e+00,0.000022,0.000081,0.000245,0.000032,0.000078,1.687107e-06,...,-0.000046,0.000010,-0.000131,-0.000009,0.000046,0.000071,0.000017,0.000073,0.000180,-0.000020
3,CD,0.667217,0.013464,0.000000e+00,0.000015,0.000005,0.000032,0.000181,0.000021,3.521316e-06,...,-0.000224,-0.000435,0.000068,-0.000016,-0.000010,0.000343,-0.000016,0.000221,0.000488,-0.000014
4,BV,0.742756,0.011865,0.000000e+00,0.000019,0.000031,0.000078,0.000021,0.000141,1.000210e-06,...,-0.000034,0.000009,0.000258,-0.000018,-0.000023,-0.000101,-0.000045,0.000153,-0.000323,-0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,CH*CD,-1.413961,0.180350,4.501759e-15,0.000049,0.000023,0.000071,0.000343,-0.000101,2.156658e-08,...,-0.000908,-0.001300,-0.016569,0.001082,0.000538,0.032526,0.002655,-0.018848,0.017392,0.001579
68,CH*BV,-0.412560,0.046379,5.824234e-19,-0.000019,0.000009,0.000017,-0.000016,-0.000045,3.070108e-07,...,-0.000013,-0.001720,-0.011916,0.000506,-0.000064,0.002655,0.002151,-0.005611,0.004517,0.000444
69,CH*CS,3.503119,0.863483,4.971465e-05,0.000442,0.000025,0.000073,0.000221,0.000153,-8.379646e-05,...,0.001165,0.019046,-0.009213,0.000555,-0.009843,-0.018848,-0.005611,0.745603,-0.072078,0.002674
70,CH*DP,-3.333008,0.457497,3.209300e-13,0.000342,0.000036,0.000180,0.000488,-0.000323,-5.085356e-06,...,-0.002342,0.036120,-0.041177,0.003112,-0.000892,0.017392,0.004517,-0.072078,0.209304,0.004062
